Selecionar apenas as colunas solicitadas pelo cliente e renomeá-las, criar coluna com a soma de todas as lesões e os dados de atualização,
filtrar estados irrelevantes e salvar os dados particionados

In [0]:
df = spark.sql("select * from ocorrencias_silver")
display(df.limit(5))

In [0]:
print(df.columns)

selecionar apenas colunas importantes

In [0]:
colunas = ['Aerodromo_de_Destino', 'Aerodromo_de_Origem','Classificacao_da_Ocorrência', 'Danos_a_Aeronave', 'Data_da_Ocorrencia','Municipio','UF','Regiao','Tipo_de_Aerodromo', 'Tipo_de_Ocorrencia','Lesoes_Desconhecidas_Passageiros', 'Lesoes_Desconhecidas_Terceiros', 'Lesoes_Desconhecidas_Tripulantes', 'Lesoes_Fatais_Passageiros', 'Lesoes_Fatais_Terceiros', 'Lesoes_Fatais_Tripulantes', 'Lesoes_Graves_Passageiros', 'Lesoes_Graves_Terceiros', 'Lesoes_Graves_Tripulantes', 'Lesoes_Leves_Passageiros', 'Lesoes_Leves_Terceiros', 'Lesoes_Leves_Tripulantes']

df = df.select(colunas)
display(df.limit(5))


Criação de coluna de soma de todas as lesões

In [0]:

colunas_a_somar = [
    "Lesoes_Desconhecidas_Passageiros",
    "Lesoes_Desconhecidas_Terceiros",
    "Lesoes_Desconhecidas_Tripulantes",
    "Lesoes_Fatais_Passageiros",
    "Lesoes_Fatais_Terceiros",
    "Lesoes_Fatais_Tripulantes",
    "Lesoes_Graves_Passageiros",
    "Lesoes_Graves_Terceiros",
    "Lesoes_Graves_Tripulantes",
    "Lesoes_Leves_Passageiros",
    "Lesoes_Leves_Terceiros",
    "Lesoes_Leves_Tripulantes"
]
df = df.withColumn("Total_Lesoes", sum(df[somartudo] for somartudo in colunas_a_somar))
display(df.limit(5))

In [0]:
print(df.columns)

In [0]:
# 3°:renomear colunas para ficar mais intuitivas para o usuario final
df = df\
    .withColumnRenamed('Aerodromo_de_Destino', 'Destino')\
    .withColumnRenamed('Aerodromo_de_Origem', 'Origem')\
    .withColumnRenamed('Classificacao_da_Ocorrência', 'Classificacao')\
    .withColumnRenamed('Danos_a_Aeronave', 'Danos')\
    .withColumnRenamed('Data_da_Ocorrencia', 'Data')\
    .withColumnRenamed('Aerodromo_de_Destino', 'Destino')\
    .withColumnRenamed('Aerodromo_de_Destino', 'Destino')
display(df.limit(5))


Removemos os registros como Estado iguais a "Indeterminado", "Sem Registro"e "Exterior"para garantir a qualidade dos dados e focar apenas em informações de ocorrências dentro do território nacional, otimizando a análise.

In [0]:

Filtro = ["Indeterminado", "Sem Registro", "Exterior"]
display( df.filter( ~df['UF'].isin(Filtro)).limit(5) )

In [0]:


classificacoes_a_excluir = ["Indeterminado", "Sem Registro", "Exterior"]
df = df.filter(~df['UF'].isin(classificacoes_a_excluir))
display(df.limit(5))


Adiciona uma coluna "Atualizacao" com a data e hora da última atualização no fuso horário de São Paulo,
permitindo que os usuários saibam quando os dados foram atualizados.

In [0]:
from pyspark.sql.functions import current_timestamp, date_format,from_utc_timestamp
df = df.withColumn("Atualizacao",\
     date_format( from_utc_timestamp(current_timestamp(), "America/Sao_Paulo"), "yyyy-MM-dd HH:mm:ss"))
display(df.limit(5))

Salvando em formato Delta, particionando e testando

In [0]:
df.write.format("delta") \
    .partitionBy("UF") \
    .mode("overwrite") \
    .saveAsTable("ocorrencias_gold")

In [0]:
display(spark.sql("select * from ocorrencias_gold where UF = 'MG'").limit(5))